# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera. **(A to już znamy!!!)**

## Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [53]:
import random, sys, os, math

def prime_naive(n):
    if n < 2:
        return False

    m = math.isqrt(n)
    for i in range(2, m + 1):
        if n % i == 0:
            return False

    return True

SIZE = 1000000
IS_PRIME = [prime_naive(n) for n in range(SIZE)]
PRIMES = [n for n in range(SIZE) if IS_PRIME[n]]
SET_OF_PRIMES = set(PRIMES)

def prime_ref(n):
    """
    Referencyjny test pierwszości, z którym porównywać będziemy inne metody.
    """
    if n < SIZE:
        return IS_PRIME[n]
    else:
        return prime_naive(n)


primes_below_2000 = 0
while PRIMES[primes_below_2000] < 2000:
    primes_below_2000 += 1

def miller_rabin_test(n, k=5):
    if n < 2:
        return False
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False

    r = 0
    s = n - 1
    while s % 2 == 0:
        r += 1
        s //= 2

    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True


def generatePrime(bits):
    while True:
        n = random.getrandbits(bits) | (1 << (bits - 1)) | 1
        for p in PRIMES[:primes_below_2000]:
            if n % p == 0 and n != p:
                break
        else:
            if miller_rabin_test(n, 5):
                return n


In [54]:
def extended_euclid(a, b):
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1
    
    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t
    
    return (old_r, old_s, old_t)

def inverse(a, n):
    a = a % n
    g, s, _ = extended_euclid(a, n)
    if g != 1:
        print(f"a and n arent coprime (gcd({a}, {n}) = {g})")
        return

    return s % n

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$ **(potrzebujemy rozszerzonego algorytmu Euklidesa!!!)**
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [55]:
def generateKey(keySize):
    p = generatePrime(keySize // 2)
    q = generatePrime(keySize // 2)
    n = p * q
    phi = (p - 1) * (q - 1)

    e = generatePrime(keySize)

    d = inverse(e, phi)

    publicKey = (e, n)
    privateKey = (d, n)
    
    return (publicKey, privateKey)

def makeKeyFiles(keySize):
    public, private = generateKey(keySize)
  
print('Generujemy klucze publiczny i prywatny')
makeKeyFiles(16)

Generujemy klucze publiczny i prywatny


## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [56]:
def encrypt(message, modulus, exp):
    return message ** exp % modulus

### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [57]:
def decrypt(message_encrypted, modulus, exp):
    return message_encrypted ** exp % modulus

## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy) - zweryfikuj jak na wydajnosć wpływa zastosowanie różnych sposobów potęgowania dostępnych w Python i jego bibliotekach.  
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


In [58]:
public_key, private_key = generateKey(16)
message = 12345

encrypted_message = encrypt(message, private_key[1], private_key[0])
decrypted_message = decrypt(encrypted_message, public_key[1], public_key[0])

print(f"Original message: {message}")
print(f"Encrypted message: {encrypted_message}")
print(f"Decrypted message: {decrypted_message}")

Original message: 12345
Encrypted message: 35250
Decrypted message: 12345


### Algorytm szybkiego potęgowania 
1. Zwykłe potęgowanie $n^{exp}$: $exp$ mnożeń 
2. Algorytm szybkiego potęgowania: część mnożeń zastępujemy podnoszeniem do kwadratu (_squaring_).
    __Skąd mamy wiedzieć kiedy mnożyć, a kiedy potęgować?__

In [59]:

def fastModularExponentation(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m

    

In [61]:
from time import time

public_key, private_key = generateKey(16)

messages = [random.getrandbits(8) for _ in range(100)]

start = time()

encrypted_messages = [encrypt(msg, private_key[1], private_key[0]) for msg in messages]
decrypted_messages = [decrypt(msg_enc, public_key[1], public_key[0]) for msg_enc in encrypted_messages]
end = time()
all_correct = all(orig == dec for orig, dec in zip(messages, decrypted_messages))
print(f"All messages decrypted correctly: {all_correct} in {end - start:.4f} seconds")


All messages decrypted correctly: True in 1.3381 seconds


In [62]:
def fast_encrypt(message, modulus, exp):
    return fastModularExponentation(message, exp, modulus)

def fast_decrypt(message_encrypted, modulus, exp):
    return fastModularExponentation(message_encrypted, exp, modulus)

public_key, private_key = generateKey(16)

messages = [random.getrandbits(8) for _ in range(100)]

start = time()
encrypted_messages = [fast_encrypt(msg, private_key[1], private_key[0]) for msg in messages]
decrypted_messages = [fast_decrypt(msg_enc, public_key[1], public_key[0]) for msg_enc in encrypted_messages]
end = time()
all_correct = all(orig == dec for orig, dec in zip(messages, decrypted_messages))
print(f"All messages decrypted correctly: {all_correct} in {end - start:.4f} seconds")



All messages decrypted correctly: True in 0.0005 seconds
